<a href="https://colab.research.google.com/github/takatakamanbou/Vision/blob/main/Vision2021_ex04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vision2021-ex04

課題の期限や提出の方法などについては，Visionチーム内に書いてます．

## はじめに

### Notebook の動かし方

この Notebook では，上の方のセルで作った変数や関数を後のセルで使うことがあります．そのため，上の方のセルを実行せずに下の方を実行すると，エラーになることがあります．上から順に実行していきましょう．

また，メニューの「ランタイム」から，「すべてのセルを実行」したりすることも可能です．

## 準備

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn
seaborn.set()

## 1次元の正規分布

1次元正規分布の確率密度関数
$$ p(x) = \frac{1}{\sqrt{2\pi\sigma^2}}\exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right) $$

上記に相当する関数 Gaussian1 を定義．自分で式を書いても大したことないが，ここでは
SciPy の関数 norm を利用: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html#scipy.stats.norm

In [ ]:
from scipy.stats import norm

def Gaussian1(x, mu, sigma2):
    return norm.pdf(x, loc = mu, scale = np.sqrt(sigma2))

1次元正規分布の確率密度関数を描画

In [ ]:
x = np.linspace(-5,5, num = 100)
fig, ax = plt.subplots(facecolor="white", figsize=(8, 6))
mu, sig2 = 0.0, 1.0
ax.plot(x, Gaussian1(x, mu, sig2), '-', label = '$\mu = {0}, \sigma^2 = {1}$'.format(mu, sig2))
mu, sig2 = 2.0, 0.5
ax.plot(x, Gaussian1(x, mu, sig2), '-', label = '$\mu = {0}, \sigma^2 = {1}$'.format(mu, sig2))
mu, sig2 = -2.0, 2.0
ax.plot(x, Gaussian1(x, mu, sig2), '-', label = '$\mu = {0}, \sigma^2 = {1}$'.format(mu, sig2))
ax.set_xlim(-5, 5)
ax.legend()
plt.show()

## 1次元正規分布の最尤推定

np.random.normal で正規乱数を生成し，np.mean, np.var で平均と分散を推定
- https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.random.normal.html
- np.mean, np.var はデフォルトでは N で割る

In [ ]:
# loc: 平均, scale: 標準偏差, size: データ数 or shape
N = 10
data = np.random.normal(loc = 0.0, scale = 1.0, size = N)
print(f'{N} {np.mean(data):.3f} {np.var(data):.3f}')
N = 100
data = np.random.normal(loc = 0.0, scale = 1.0, size = N)
print(f'{N} {np.mean(data):.3f} {np.var(data):.3f}')
N = 1000
data = np.random.normal(loc = 0.0, scale = 1.0, size = N)
print(f'{N} {np.mean(data):.3f} {np.var(data):.3f}')

上記と同様の実験，ただし結果をグラフに描く．N をいろいろ変えて実行し直してみよう．同じ N でも実行のたびに乱数の値が変わるので，何度か実行して様子を眺めるのがよい．

In [ ]:
# 真のパラメータ
mu_true, sigma_true = 0.0, 1.0
print('# mu_true = {0:.2f}, sigma_true = {1:.2f}'.format(mu_true, sigma_true))

# 上記のパラメータをもつ1次元正規分布から標本を抽出
N = 10
data = np.random.normal(loc = mu_true, scale = sigma_true, size = N)

# 上記の標本から正規分布のパラメータを最尤推定
mu_est, sigma_est = np.mean(data), np.sqrt(np.var(data))
print('# mu_est = {0:.2f}, sigma_est = {1:.2f}'.format(mu_est, sigma_est))

# グラフ
fig, ax = plt.subplots(2, facecolor="white", figsize=(8, 12))

# 標本のヒストグラム
ax[0].hist(data, density = True)

# 真の正規分布と推定された正規分布
x = np.linspace(-5,5, num = 100)
#fig, ax = plt.subplots(facecolor="white", figsize=(8, 6))
ax[1].plot(x, Gaussian1(x, mu_true, sigma_true**2), '-', label = 'true')
ax[1].plot(x, Gaussian1(x, mu_est, sigma_est**2), '-', label = 'estimated')
ax[1].set_xlim(-5, 5)
ax[1].legend()
plt.show()

## 多変量正規分布
$D$次元正規分布の確率密度関数
$$ p(\mathbf{x}) = \frac{1}{\sqrt{(2\pi)^D |\Sigma|}} \exp\left( -\frac{1}{2}(\mathbf{x} - \mathbf{\mu})^{\top}\Sigma^{-1}(\mathbf{x} - \mathbf{\mu}) \right)$$
$\mathbf{\mu}$ は平均ベクトル，$\Sigma$ は分散共分散行列．

上記に相当する関数 Gaussian を定義．自分で式を書いても大したことないが，ここでは
SciPy の関数 multivariate_normal を利用: 

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.multivariate_normal.html

In [ ]:
from scipy.stats import multivariate_normal

def GaussianM(X, mean, cov):
    return multivariate_normal.pdf(X, mean = mean, cov = cov)

In [ ]:
x, y = np.mgrid[-3:3:0.01, -3:3:0.01]
X = np.dstack((x, y))
print(X.shape)

$\mathbf{\mu} = \mathbf{0}, \Sigma = I$ のとき

In [ ]:
# グラフ描画用のグリッドデータの作成
xmin, xmax = -4, 4
ymin, ymax = -4, 4
x, y = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X = np.dstack((x, y))

# 平均と共分散行列
mean = np.zeros(2)
cov = np.eye(2)

# グラフを描く
fig, ax = plt.subplots(facecolor="white", figsize=(8, 8))
ax.contourf(x, y, GaussianM(X, mean , cov))
ax.axhline(0, color='white')
ax.axvline(0, color='white')
ax.set_xlim(xmin,xmax)
ax.set_ylim(ymin,ymax)
ax.set_aspect('equal')
plt.show()

$\mathbf{\mu} = \begin{pmatrix} 2 \\ 1 \end{pmatrix} , \Sigma = \begin{pmatrix} 2 & 0 \\ 0 & \frac{1}{2} \end{pmatrix}$ のとき

In [ ]:
# グラフ描画用のグリッドデータの作成
xmin, xmax = -3, 5
ymin, ymax = -3, 5
x, y = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X = np.dstack((x, y))

# 平均と共分散行列
mean = np.array([2, 1])
cov = np.array([[2, 0], [0, 0.5]])

# グラフを描く
fig, ax = plt.subplots(facecolor="white", figsize=(8, 8))
ax.contourf(x, y, GaussianM(X, mean , cov))
ax.axhline(0, color='white')
ax.axvline(0, color='white')
ax.set_xlim(xmin,xmax)
ax.set_ylim(ymin,ymax)
ax.set_aspect('equal')
plt.show()

In [ ]:
# 回転行列
def R(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s], [s, c]])

$\mathbf{\mu} = \begin{pmatrix} 2 \\ 1 \end{pmatrix}  , \Sigma = R(\pi/3)\begin{pmatrix} 2 & 0 \\ 0 & \frac{1}{2} \end{pmatrix} R(\pi/3)^{\top} = \begin{pmatrix}\frac{1}{2} & -\frac{\sqrt{3}}{2} \\ \frac{\sqrt{3}}{2} & \frac{1}{2} \end{pmatrix}\begin{pmatrix} 2 & 0 \\ 0 & \frac{1}{2} \end{pmatrix}\begin{pmatrix}\frac{1}{2} & \frac{\sqrt{3}}{2} \\ -\frac{\sqrt{3}}{2} & \frac{1}{2} \end{pmatrix} = \begin{pmatrix} \frac{7}{8} & \frac{3}{8}\sqrt{3} \\ \frac{3}{8}\sqrt{3} & \frac{13}{8}\end{pmatrix}$ のとき

In [ ]:
# グラフ描画用のグリッドデータの作成
xmin, xmax = -3, 5
ymin, ymax = -3, 5
x, y = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X = np.dstack((x, y))

# 平均と共分散行列
mean = np.array([2, 1])
cov = np.array([[7.0/8, 3*np.sqrt(3)/8], [3*np.sqrt(3)/8, 13.0/8]])
print(cov)
cov = R(np.pi/3) @ np.diag([2, 0.5]) @ R(np.pi/3).T
print(cov)

# グラフを描く
fig, ax = plt.subplots(facecolor="white", figsize=(8, 8))
ax.contourf(x, y, GaussianM(X, mean , cov))
ax.axhline(0, color='white')
ax.axvline(0, color='white')
ax.set_xlim(xmin,xmax)
ax.set_ylim(ymin,ymax)
ax.set_aspect('equal')
plt.show()

これは，$\mathbf{\mu} = \mathbf{0} , \Sigma = \begin{pmatrix} 2 & 0 \\ 0 & \frac{1}{2} \end{pmatrix}$ の正規分布を $\theta$ 回転して $\mathbf{\mu}$ 平行移動したものになっている．

## 多変量正規分布の最尤推定

In [ ]:
# 2次元正規分布のデータ生成（標準正規分布の乱数から自分で回転平行移動）

def gendat(N):
        
    # 標準正規分布からとったサンプルを 2 x N にならべる
    x = np.random.normal(loc = 0.0, scale = 1.0, size = (2, N))
    # \Sigma^{\frac{1}{2}} をかけて共分散行列 \Sigma のサンプルにする
    y = np.diag([np.sqrt(2), np.sqrt(0.5)]) @ x
    # R(theta) 回転 して mu 平行移動
    z =  R(np.pi/3) @ y + np.array([2.0, 1.0])[:, np.newaxis]
    
    return z

In [ ]:
# 2次元正規分布のデータ生成（np.random.multivariate_normal を使う）

def gendat2(N):
    
    mu = np.array([2.0, 1.0])
    cov = R(np.pi/3) @ np.diag([2, 0.5]) @ R(np.pi/3).T
    x = np.random.multivariate_normal(mu, cov, size = N)  #  N x 2

    return x.T  # 2 x N

In [ ]:
# 真の平均，共分散行列
mu_true = np.array([2.0, 1.0])
cov_true = R(np.pi/3) @ np.diag([2, 0.5]) @ R(np.pi/3).T
print('### 真の平均と共分散行列')
print(mu_true)
print(cov_true)
print()

for N in [10, 100, 1000, 10000]:
    print('### N = {0} での平均と共分散行列の推定値'.format(N))
    data = gendat2(N)
    mu_est = np.mean(data, axis = 1)
    x = data - mu_est[:, np.newaxis]
    cov_est = x @ x.T / N
    print(mu_est)
    print(cov_est)
    print()

上記と同様の実験．ただしグラフを描く．N をいろいろ変えて実行し直してみよう．同じ N でも実行のたびに乱数の値が変わるので，何度か実行して様子を眺めるのがよい．

In [ ]:
#  データの生成
N = 10
data = gendat2(N)

# 平均，共分散行列の推定
mu_est = np.mean(data, axis = 1)
x = data - mu_est[:, np.newaxis]
cov_est = x @ x.T / N

# グラフ描画用のグリッドデータの作成
xmin, xmax = -3, 5
ymin, ymax = -3, 5
x, y = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X = np.dstack((x, y))

# グラフを描く
fig, ax = plt.subplots(facecolor="white", figsize=(8, 8))
ax.scatter(data[0, :], data[1, :], s = 10)
ax.contour(x, y, GaussianM(X, mu_est , cov_est), colors = ['#ffa0a0', 'r'], levels = [0.05, 0.1])  # 推定された正規分布
ax.contour(x, y, GaussianM(X, mu_true , cov_true), colors = ['#a0ffa0', 'g'], levels = [0.05, 0.1])  # 真の正規分布
ax.set_aspect('equal')
plt.show()